##### Copyright 2023 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PaLM API Quickstart - Text Edition

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this notebook, you'll learn how to get started with the PaLM API, which gives you access to Google's latest large language models. Here, you'll learn how to use the PaLM API's text generation features.

## Setup

**Note**: At this time, the PaLM API is only available in the US.

First, download and install the PaLM API Python library.

In [ ]:
!pip install -q google-generativeai

In [ ]:
import pprint
import google.generativeai as palm

### Grab an API Key

To get started, you'll need to [create an API key](https://developers.generativeai.google/tutorials/setup).

In [ ]:
palm.configure(api_key='PALM_KEY')

## Text generation

Use the `palm.list_models` function to find available models:

In [ ]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


Use the `palm.generate_text` method to generate text:

In [ ]:
prompt = """
I have three houses, each with three cats.
each cat owns 4 mittens, and a hat. Each mitten was
knit from 7m of yarn, each hat from 4m.
How much yarn was needed to make all the items?

Think about it step by step, and show your work.
"""

completion = palm.generate_text(
    model=model,
    prompt=prompt,
    # The maximum length of the response
    max_output_tokens=800,
)

print(completion.result)

There are 3 houses * 3 cats / house = 9 cats.
There are 9 cats * 4 mittens / cat = 36 mittens.
There are 9 cats * 1 hat / cat = 9 hats.
The total amount of yarn needed for the mittens is 36 mittens * 7m / mitten = 252m.
The total amount of yarn needed for the hats is 9 hats * 4m / hat = 36m.
The total amount of yarn needed is 252m + 36m = 288m.
So the answer is 288


## More options

Typically, there's some degree of randomness in the text produced by LLMs. (Read more about why in the [LLM primer](https://developers.generativeai.google/guide/concepts)). That means that when you call the API more than once with the same input, you might get different responses. You can use this feature to your advantage to get alternate model responses.

The `temperature` argument controls the variance of the responses. The `palm.Model` object gives the default value for `temperature` and other parameters.

In [ ]:
models[0]

Model(name='models/text-bison-001', base_model_id='', version='001', display_name='Text Bison', description='Model targeted for text generation.', input_token_limit=4000, output_token_limit=1024, supported_generation_methods=['generateText'], temperature=0.7, top_p=0.95, top_k=40)

The `candidate_count` argument controls the number of responses returned:

In [ ]:
completion = palm.generate_text(
    model=model,
    prompt=prompt,
    # The number of candidates to return
    candidate_count=8,
    # Set the temperature to 1.0 for more variety of responses.
    temperature=1.0,
    max_output_tokens=800,
)

print(completion.result)

There are 3 * 3 = 9 cats. 9 cats have 9 * 4 = 36 mittens. 36 mittens are made of 36 * 7 = 252m of yarn. There are also 9 hats which require 9 * 4 = 36m of yarn. Therefore, a total of 252 + 36 = 288m of yarn was used.
So the answer is 288


When you request multiple candidates the `Completion.result` attribute still just contains the first one. The `Completion.candidates` attribute contains all of them:

In [ ]:
import pprint
pprint.pprint(completion.candidates)

[{'output': 'There are 3 * 3 = 9 cats. 9 cats have 9 * 4 = 36 mittens. 36 '
            'mittens are made of 36 * 7 = 252m of yarn. There are also 9 hats '
            'which require 9 * 4 = 36m of yarn. Therefore, a total of 252 + 36 '
            '= 288m of yarn was used.\n'
            'So the answer is 288'},
 {'output': 'There are 3 houses * 3 cats = 9 cats.\n'
            'So, there were 9 cats * 4 mittens = 36 mittens.\n'
            'And there were 9 cats * 1 hat = 9 hats.\n'
            'The mittens needed 36 mittens * 7m of yarn = 252m of yarn.\n'
            'And the hats needed 9 hats * 4m of yarn = 36m of yarn.\n'
            'So in total, 252m + 36m = 288m of yarn were needed.\n'
            'So the answer is 288'},
 {'output': 'There are a total of 3 * 3 = 9 cats.\n'
            'Each cat has 4 mittens so there are 9 * 4 = 36 mittens.\n'
            'Each hat uses 4 meters of yarn so all the hats take 36 * 4 = 144 '
            'meters of yarn.\n'
            'Each mitte

In [ ]:
import numpy as np
np.mean(['288' in c['output'] for c in completion.candidates])

0.875